# Managing memory

## Variable names are just labels

To start, note that variables in Python are just names or labels for objects. Think of them as post-it notes on objects instead of as a predefined box that you put data into, as with statically typed languages:

![](extras/sticky-note.jpg)

In [56]:
a = 'Me'
b = a
a = 'Myself'
a, b

('Myself', 'Me')

## Mutability

Python offers both mutable and immutable data types. Those that are "immutable" cannot be changed once created.

Mutable data types offer the advantage of convenience of use in imperative programming constructs like ``for`` loops.

Immutable data types offer the advantage of safety; an immutable object cannot be corrupted by passing it into a badly written function. Immutable objects are also inherently thread-safe; there is no need to lock something that cannot change. The immutability also makes it possible to share immutable objects more cheaply.

There are performance advantages and disadvantages to mutability. Operations that mutate an object, like string concatenation, can sometimes be done with less memory copying with mutable types. On the other hand, operations that share an object, like passing it to a function, benefit from reduced memory consumption and improved cache utilisation with immutable types.

Perhaps more important than performance is the difference in programming style that immutable types enforces: safer and more functional, and hence more parallelizable.

## Mutable and immutable types

| Immutable | Mutable |
|-|-|
| tuple | list |
| bytes | bytearray |
| str | &nbsp; |
| frozenset | set |
| frozendict | dict |
| range | &nbsp; |
| int | &nbsp; |
| float | &nbsp; |
| complex | &nbsp; |

Alternatives for a mutable string include a ``list`` of strings and ``io.StringIO``, which is an in-memory file-like stream for text IO.

In [5]:
s = frozenset(['a', 'b', 'a', 'c'])

In [8]:
print(dir(s))

['__and__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__rand__', '__reduce__', '__reduce_ex__', '__repr__', '__ror__', '__rsub__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__xor__', 'copy', 'difference', 'intersection', 'isdisjoint', 'issubset', 'issuperset', 'symmetric_difference', 'union']


This raises a TypeError:

In [9]:
del s['a']

TypeError: 'frozenset' object does not support item deletion

## object ``id``

The ``id()`` function returns a unique numerical ID for each named object. If two variable names have the same ``id()``, they refer to the same object:

In [1]:
a = 1
id(a)

4297327232

In [2]:
b = a

In [4]:
id(b)

4297327232

In [7]:
def double(myvar):
    print(id(myvar))
    myvar *= 2
    print(id(myvar))

In [8]:
double(a)

4297327232
4297327264


In [9]:
id(a)

4297327232

The `id` of `a` has not changed. This is because `a` is an integer and integers are immutable.

Contrast what happens when we pass a mutable object like a list into `double`:

In [18]:
names = ['Barney', 'Betty', 'Bambam']

In [19]:
double(names)

4394620744
4394620744


In [20]:
names

['Barney', 'Betty', 'Bambam', 'Barney', 'Betty', 'Bambam']

Strings are also immutable:

In [12]:
s = 'abc'
id(s)

4338148216

In [13]:
s += 'def'
id(s)

4395430328

You see the above has actually sneakily created a new string object `s`. The ID has changed.

Because integers and strings are immutable (see the table above), Python "fakes" operations like ``+=`` by reassigning the variable name to a new object with the new value. Another example:

In [25]:
a += 1
a

2

In [26]:
id(a)

4297326656

In [28]:
id(a) == id(b)

False

``a`` refers to a different object now (``2``), but ``b`` still refers to the same one (``1``).

In [29]:
b

1

In [20]:
id(b)

4297326624

The same is true with strings:

In [30]:
s = 'Hello'
id(s)

4416281768

In [31]:
s += ' world'
id(s)

4416185840

This is why string concatenation is relatively slow in Python and why ``str.join()`` is recommended instead.

### Comparison with other languages

The following languages have immutable strings:
- Python
- Java
- C#
- Haskell
- Clojure
- Scala

## Pass-by-reference or pass-by-value?

Python passes both immutable and mutable objects by reference, but not in the same sense as in Java. Passing a variable has the effect as passing the ``id``s of the objects.

In [39]:
i = 1
s = 'Hello'
l = ['Item 1', 'Item 2']

def f(i, s, l):
    return (i, s, l)

(i2, s2, l2) = f(i, s, l)

print('Same object?')
print('i:', id(i) == id(i2))
print('s:', id(s) == id(s2))
print('l:', id(l) == id(l2))

Same object?
i: True
s: True
l: True


### Question
Is it possible to write a ``swap(a, b)`` function in Python that swaps the values of ``a`` and ``b`` in-place?

### Answer
Yes, but only if ``a`` and ``b`` are mutable.

### Exercise

1. Write such a function for lists.

In [ ]:
a, b = b, a

One solution:

In [34]:
def swap(a, b):
    c = a.copy()
    d = b
    a[:] = b
    b[:] = c

In [35]:
names1 = ['Fred', 'Betty']
print(id(names1))
names2 = ['Wilma', 'Barney']
print(id(names2))

4395242248
4395461960


In [36]:
swap(names1, names2)

In [37]:
names1

['Wilma', 'Barney']

In [38]:
names2

['Fred', 'Betty']

2. Try writing such a function for tuples.


### Exercise: Can you explain this?

In [43]:
def reassign(list):
    list = [0, 1]

list = [0]
reassign(list)
print(list)

[0]


The line ``list = [0, 1]`` inside the function creates a local variable (name) which shadows the global one within the scope of the function.

Try printing the ``id`` of ``list`` outside and inside the function.

In [44]:
def append(list):
    list.append(1)

append(list)
print(list)

[0, 1]


## Garbage collection

Python's memory is automatically allocated and freed.

The garbage-collection implementation varies among different Python interpreters (CPython, PyPy, ...).

The CPython interpreter uses both reference-counting and scheduled garbage collection. When no more references point to an object, its memory is freed. This is simple and efficient, provided that there are no "reference cycles" (two objects containing references to each other). To the extent possible, it is good practice to write code without reference cycles.

CPython's scheduled garbage collector takes time to run, so CPython schedules garbage collection periodically -- when the difference between the number of allocations and deallocations exceeds a threshold:

In [61]:
import gc
gc.get_threshold()

(700, 10, 10)

The threshold here is 700 allocations. (The three values refer to thresholds for 3 generations. See the ``gc`` docs for more details.)

### Manual garbage collection

In [62]:
import gc
gc.collect()

335

The value returned is the number of objects it has collected and deallocated.

For full details of CPython's garbage collector, see: http://hg.python.org/cpython/file/tip/Modules/gcmodule.c.

### When to use manual garbage collection

If you are seeing memory leaks with long-running server processes, try invoking ``gc.collect()`` periodically.

If responsiveness is a problem (e.g. for a server that must respond quickly to requests), you can turn off garbage collection with ``gc.disable()`` and then either:

- manually collect when the system is idle
- configure ``mod_wsgi`` to kill and restart processes more frequently.

### Garbage collection statistics

To find out more, use:

In [63]:
gc.set_debug(gc.DEBUG_STATS)

You can also find places that require attention with:

In [67]:
a = []
b = [a]
a.append(a)
referrers = gc.get_referrers(a)